In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import tqdm
import sys
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [2]:
# read csv from player_info
df = pd.read_csv('player_info.csv')

In [3]:
# create "name" column
df['name'] = df['firstName'] + " " + df['lastName']
df.head()

,player_id,firstName,lastName,nationality,birthCity,primaryPosition,birthDate,link,name
0,8467412,Alexei,Ponikarovsky,UKR,Kiev,LW,1980-04-09,/api/v1/people/8467412,Alexei Ponikarovsky
1,8468501,Anton,Volchenkov,RUS,Moscow,D,1982-02-25,/api/v1/people/8468501,Anton Volchenkov
2,8459670,Kimmo,Timonen,FIN,Kuopio,D,1975-03-18,/api/v1/people/8459670,Kimmo Timonen
3,8471233,Travis,Zajac,CAN,Winnipeg,C,1985-05-13,/api/v1/people/8471233,Travis Zajac
4,8455710,Martin,Brodeur,CAN,Montreal,G,1972-05-06,/api/v1/people/8455710,Martin Brodeur


In [4]:
# scrap data by beautiful soup 
browser = Browser('chrome', headless = True)

In [5]:
headlines = []

In [6]:
browser.visit('https://www.reuters.com/news/archive/icehockey-nhl?view=page&page=1&pageSize=10')

In [7]:
soup = BeautifulSoup(browser.html, 'html.parser')

In [8]:
hls = soup.find_all(class_ = "story-title")
hls[3].text.replace("\n", "").replace('\t', "")

'Rask, Bruins close out Jackets in Game 6'

In [9]:

for i in range(len(hls)):
    headlines.append(hls[i].text.replace("\n", "").replace('\t', ""))


In [10]:
browser.click_link_by_partial_text('Earlier')

In [11]:
for i in tqdm.tqdm(range(1000)):
    soup = BeautifulSoup(browser.html, 'html.parser')
    hls = soup.find_all(class_ = "story-title")
    for i in range(len(hls)):
        headlines.append(hls[i].text.replace("\n", "").replace('\t', ""))
    try:
        browser.click_link_by_partial_text('Earlier')
    except Exception as e:
        print(e)
        break

  2%|▏         | 24/1000 [01:12<47:32,  2.92s/it] 

Message: timeout
  (Session info: headless chrome=74.0.3729.131)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Mac OS X 10.14.4 x86_64)



In [12]:
len(headlines)

338

In [13]:
# with open("headlines.txt", "w") as f:
#     for s in headlines:
#         f.write(str(s) +"\n")

In [14]:
# convert dataframe headlines to csv file and save it
headlines_df = pd.DataFrame(data=temp)
headlines_df.to_csv('headlines.csv')

NameError: name 'temp' is not defined

In [ ]:
temp = []

In [ ]:
with open("headlines.txt", "r") as f:
    for line in f:
        temp.append(line.strip())

In [ ]:
temp.pop(383)

In [ ]:
temp[383]

In [ ]:
temp.pop(383)

In [ ]:
df

In [ ]:
df['headlines'] = '0'

In [ ]:
headlines_list = temp

In [ ]:
player_count = 0
for i in tqdm.tqdm(range(len(df))):
    last_name = df.iloc[i, 2]
    for j in range(len(headlines_list)):
        if last_name in headlines_list[j]:
            player_count += 1
    df.iat[i,9] = int(df.iat[0, 9]) + player_count
    player_count = 0

In [ ]:
df.iat[0, 9] = int(df.iat[0, 9]) + 1

In [ ]:
df.to_csv('players_headlines.csv')

In [ ]:
if 'Williams' in 'Williams, Hurricanes top Islanders for 3-0 series lead':
    player_count += 1

In [ ]:
player_count

In [ ]:
headlines_list

In [ ]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()

In [ ]:
conn = create_connection("nhl_info.db")

In [ ]:
# create database path, engin and connection
database_path = "nhl_info.db"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()
# Base = automap_base()
# Base.classes.keys()

In [ ]:
engine.execute(
'CREATE TABLE players (
    player_id = Column(Integer, primary_key=True)
    firstName = Column(String(255))
    lastName = Column(String(255))
    nationality = Column(String(255))
    birthCity = Column(String(255))
    primaryPosition = Column(String(255))
    birthDate = Column(String(255))
    link = Column(String(255))
    name = Column(String(255))

)
'')

In [ ]:
# create classes and columns for the new tables ( palyers & headlines ) in new SQL database 
class Players(Base):
    __tablename__ = 'players'
    player_id = Column(Integer, primary_key=True)
    firstName = Column(String(255))
    lastName = Column(String(255))
    nationality = Column(String(255))
    birthCity = Column(String(255))
    primaryPosition = Column(String(255))
    birthDate = Column(String(255))
    link = Column(String(255))
    name = Column(String(255))

class Headlines(Base):
    __tablename__ = 'headlines'
    id = Column(Integer, primary_key=True)
    headline = Column(String(3948))

In [ ]:
Players = Base.classes.players
Headlines = Base.classes.headlines

In [ ]:
# get data back from csv files 
players = pd.read_csv('player_info.csv')
headlines = pd.read_csv('headlines.csv')
players_headlines = pd.read_csv('players_headlines.csv')

In [ ]:
type(players)

In [ ]:
print(len(players))

In [ ]:
players_headlines.head()

In [ ]:
headlines.head()

In [ ]:
# Convert pandas dataframes to sql files 
players.to_sql('players',conn)
headlines.to_sql('headlines',conn)
players_headlines.to_sql('players_headlines',conn)

In [ ]:
test = engine.execute('SELECT * FROM players_headlines where firstName = "Daniel"').fetchall()
test

In [ ]:
# Create a Specific Instance of the Players and Headlines classes
# ----------------------------------
# dog = Dog(name="Fido", color='Brown', age=4)
# cat = Cat(name="Whiskers", color="Gray", age=7)
for player in players.rows:
    print(player)

In [ ]:
for headline in headlines:
    print(headline)

In [ ]:
# Create Database Connection
# ----------------------------------
conn = engine.connect()

In [ ]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

In [ ]:
# Create a Session Object to Connect to DB
# ----------------------------------
from sqlalchemy.orm import Session
session = Session(bind=engine)